In [1]:
#ライブラリインポート
import pandas as pd
import matplotlib.pyplot as plt
import gensim
from janome.tokenizer import Tokenizer
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors
from sklearn.cluster import KMeans
from gensim.models import word2vec
import numpy as np
from sklearn.linear_model import LinearRegression
import sys
import joblib

In [37]:
# 集めた歌詞データを読み込む（別で取得）
df_concat = pd.read_excel("usedata.xlsx",engine="openpyxl")
df_concat = df_concat.drop(columns="Unnamed: 0")
df_concat = df_concat.sort_values("release_date")
# 1986年1月〜2023年8月
lyrics_list = df_concat["lyrics"][267:322393].tolist()
release_date_list = df_concat["release_date"][267:322393].tolist()

In [51]:
songs_list_4sq = []
for i in range(38):
    a = []
    for j in range(4):
        a.append([])
    songs_list_4sq.append(a)
# 文字列が多すぎるため3次元配列として持てないので配列のインデックス番号で保持。
for i, j in enumerate(release_date_list):
    year = int(j[:4])
    month = int(j[4:])
    year_num = year - 1986
    month_num = (month-1)//3
    songs_list_4sq[year_num][month_num].append(i)

In [54]:
# モデル1データセット 1986〜2000年
model1_songs_index = songs_list_4sq[:14]
model1_songs_list = lyrics_list[:34296]
# モデル2データセット 2000〜2023年
model2_songs_index = songs_list_4sq[14:]
model2_songs_list = lyrics_list[34296:]
# モデル3データセット 1986〜2023年
model3_songs_index = songs_list_4sq
model3_songs_list = lyrics_list

model1_text_file = 'model1_songs_word_lists.txt'
model2_text_file = 'model2_songs_word_lists.txt'
model3_text_file = 'model3_songs_word_lists.txt'

In [42]:
def data_processing(song_lyrics, text_file):
    t = Tokenizer()
    results = []
    for s in song_lyrics:
        tokens = t.tokenize(s)
        r = []
        for tok in tokens:
            if tok.base_form == '*':
                word = tok.surface
            else:
                word = tok.base_form
            ps = tok.part_of_speech
            hinshi = ps.split(',')[0]
            if hinshi in ['名詞', '形容詞', '動詞', '副詞']:
                r.append(word)
        rl = (' '.join(r)).strip()
        results.append(rl)
    #余計な文字コードの置き換え
    result = [i.replace('\u3000','') for i in results]
    with open(text_file, 'w', encoding='utf-8') as fp:
        fp.write("\n".join(result))

In [79]:
data_processing(model1_songs_list, model1_text_file)
data_processing(model2_songs_list, model2_text_file)
data_processing(model3_songs_list, model3_text_file)

In [57]:
for i, j in enumerate(songs_list_4sq):
    i = i + 1986
    for k, l in enumerate(j):
        k = k + 1
        text_file_name = "4sq_songs/" + str(i) + str(k) + "_songs_word_lists.txt"
        t = Tokenizer()
        results = []
        for s in l:
            tokens = t.tokenize(lyrics_list[s])
            r = []
            for tok in tokens:
                if tok.base_form == '*':
                    word = tok.surface
                else:
                    word = tok.base_form
                ps = tok.part_of_speech
                hinshi = ps.split(',')[0]
                if hinshi in ['名詞', '形容詞', '動詞', '副詞']:
                    r.append(word)
            rl = (' '.join(r)).strip()
            results.append(rl)
        #余計な文字コードの置き換え
        result = [p.replace('\u3000','') for p in results]
        with open(text_file_name, 'w', encoding='utf-8') as fp:
            fp.write("\n".join(result))

In [4]:
model1_save_bin = 'model1_save.bin'
model2_save_bin = 'model2_save.bin'
model3_save_bin = 'model3_save.bin'

In [4]:
def model_learning(text_file, save_bin):
    sentences = word2vec.Text8Corpus(text_file)
    model = word2vec.Word2Vec(sentences, size=200, window=10, hs=0, min_count=1, sg=1)
    model.save(save_bin)

In [8]:
model_learning(model1_text_file, model1_save_bin)
model_learning(model2_text_file, model2_save_bin)
model_learning(model3_text_file, model3_save_bin)

In [45]:
# 1986年から2020年までを学習として2020年から2023年までをテストとする（乖離度の確認）
def analyze(save_bin:str):
    model = gensim.models.Word2Vec.load(save_bin)
    commit = []
    for i in range(34):
        i = i + 1986
        for j in range(4):
            result = []
            j = j + 1
            text_name = '4sq_songs/' + str(i) + str(j) + '_songs_word_lists.txt'
            f = open(text_name, 'r', encoding='UTF-8')
            data = f.read()
            for d in data:
                try:
                    r = model.wv.get_vector(d)
                except:
                    try:
                        p = model.wv.most_similar(positive=[d][0][0])
                        r = model.wv.get_vector(p)
                    except:
                        continue
                result.append(r)
            ans = []
            for k in range(200):
                sum_k = 0
                for l in result:
                    sum_k = sum_k + l[k]
                sum_k = sum_k/len(result)
                ans.append(sum_k)
            commit.append(ans)
    return commit

In [46]:
# 2023年は第二四半期まで
def analyze_test(save_bin:str):
    model = gensim.models.Word2Vec.load(save_bin)
    commit = []
    for i in range(4):
        i = i + 2020
        for j in range(4):
            result = []
            j = j + 1
            if i == 2023 and j == 3:
                return commit
            text_name = '4sq_songs/' + str(i) + str(j) + '_songs_word_lists.txt'
            f = open(text_name, 'r', encoding='UTF-8')
            data = f.read()
            for d in data:
                try:
                    r = model.wv.get_vector(d)
                except:
                    try:
                        p = model.wv.most_similar(positive=[d][0][0])
                        r = model.wv.get_vector(p)
                    except:
                        continue
                result.append(r)
            ans = []
            for k in range(200):
                sum_k = 0
                for l in result:
                    sum_k = sum_k + l[k]
                sum_k = sum_k/len(result)
                ans.append(sum_k)
            commit.append(ans)

In [49]:
model1_explain = analyze(model1_save_bin)
model2_explain = analyze(model2_save_bin)
model3_explain = analyze(model3_save_bin)

In [50]:
model1_explain_test = analyze_test(model1_save_bin)
model2_explain_test = analyze_test(model2_save_bin)
model3_explain_test = analyze_test(model3_save_bin)

In [84]:
joblib.dump(model1_explain, "model1_explain.txt",compress=3)
joblib.dump(model2_explain, "model2_explain.txt",compress=3)
joblib.dump(model3_explain, "model3_explain.txt",compress=3)
joblib.dump(model1_explain_test, "model1_explain_test.txt",compress=3)
joblib.dump(model2_explain_test, "model2_explain_test.txt",compress=3)
joblib.dump(model3_explain_test, "model3_explain_test.txt",compress=3)

['model3_explain_test.txt']

In [2]:
model1_explain = joblib.load("model1_explain.txt")
model2_explain = joblib.load("model2_explain.txt")
model3_explain = joblib.load("model3_explain.txt")
model1_explain_test = joblib.load("model1_explain_test.txt")
model2_explain_test = joblib.load("model2_explain_test.txt")
model3_explain_test = joblib.load("model3_explain_test.txt")

In [4]:
# 日本CPI
df_cpi = pd.read_excel("purpose_data/cpi.xlsx", engine="openpyxl", index_col=0)
# 年月日
cpi_year = df_cpi["Unnamed: 8"][13:655]
# 値
cpi_value = df_cpi.iloc[13:655,12]

In [5]:
# 1986年第1四半期〜2023年第2四半期
cpi_4sq_value = []
for i, j in enumerate(cpi_year[192:]):
    if i%3 == 0:
        a = cpi_value[192+i]
    elif i%3 == 1:
        a += cpi_value[192+i]
    else:
        a += cpi_value[192+i]
        a = a/3
        cpi_4sq_value.append(a)

In [6]:
# 1986~2020
cpi_1986_2020 = cpi_4sq_value[:136]
cpi_2020_2023 = cpi_4sq_value[136:]

In [7]:
# 日本経済指標(CI値)
# 1985年から月ベースでの調査
df_japan_index = pd.read_excel("purpose_data/economy_index.xlsx", engine="openpyxl", index_col=0)
# 年
japan_index_year = df_japan_index["Unnamed: 1"][77:527]
# 月
japan_index_month = df_japan_index["Unnamed: 2"][77:527]
# 値
japan_index_value = df_japan_index["ＣＩ指数"][77:527].tolist()

In [8]:
# 1986年第1四半期〜2023年第2四半期
japan_index_4sq_value = []
for i, j in enumerate(japan_index_value):
    if i%3 == 0:
        a = j
    elif i%3 == 1:
        a += j
    else:
        a += j
        a = a/3
        japan_index_4sq_value.append(a)

In [9]:
ci_1986_2020 = japan_index_4sq_value[:136]
ci_2020_2023 = japan_index_4sq_value[136:]

In [10]:
# 日本GDP
df_gdp = pd.read_excel("purpose_data/GDP_4sq.xlsx", engine="openpyxl")
# 年四半期
gdp_4sq = df_gdp["name1"][7:124]
# 値
gdp_4sq_value = df_gdp["name2"][7:124]
gdp_list = []
for p in gdp_4sq_value:
    gdp_list.append(p)

In [11]:
gdp_4sq_1994_2020 = gdp_list[:104]
gdp_4sq_2020_2023 = gdp_list[104:]

## 重回帰分析

In [94]:
# モデル別に実装していく
value_list = model3_explain[32:]
value_list_T = np.array(value_list).T

# 目的変数となるものを指定して実装していく
y = gdp_4sq_1994_2020

In [95]:
# 正規化
# 標準化
# 最大最小はすでに決まっているので標準化のみ
std_value_list_T = []
for i in value_list_T:
    a = (i-i.mean())/i.std(ddof=0)
    std_value_list_T.append(a)

In [96]:
model_lr_std = LinearRegression()
x = np.array(std_value_list_T).T
model_lr_std.fit(x, y)
# print(model_lr_std.coef_)
# print(model_lr_std.intercept_) 
# print(model_lr_std.score(x, y))

LinearRegression()

In [97]:
# テストモデルの実装
# ここも対応するテストに応じて変更して実装していく
test_result = []
for i in model3_explain_test[:-1]:
    a = 0
    for p in range(200):
        a += i[p]*model_lr_std.coef_[p]
    a += model_lr_std.intercept_
    test_result.append(a)

In [98]:
deviation = []
for i,j in enumerate(test_result):
    deviation.append(gdp_4sq_2020_2023[i]-j)

In [99]:
deviation

[4808.6168024968065,
 -6238.309835508859,
 -1689.7084443637868,
 10396.448295799346,
 3191.194622056297,
 2764.1955705721048,
 291.76371169951744,
 11038.964048447378,
 4427.972447120206,
 4800.024070856365,
 1907.1823786955792,
 13303.324628000206,
 9803.104250009754]

In [100]:
result = []
for i in model_lr_std.coef_.tolist():
    result.append(round(i, 5))

In [101]:
result

[2177.29428,
 2288.55286,
 -2933.23373,
 -1881.71315,
 2484.7671,
 -10059.22943,
 -1059.28857,
 -138.23649,
 6460.15915,
 1420.47938,
 -4576.51039,
 -2613.80593,
 -591.80417,
 -4776.57951,
 528.07046,
 -5962.30227,
 8496.07597,
 4314.24381,
 -1233.11173,
 9989.11574,
 -2888.51874,
 6394.51973,
 -2572.41079,
 6814.60768,
 -1053.85117,
 3204.35052,
 -361.59339,
 -1331.51017,
 -4402.69813,
 -272.17946,
 -391.48958,
 -2424.54671,
 -4083.73266,
 -1071.41555,
 2143.63742,
 -6613.82086,
 -2706.75166,
 -8361.81806,
 9555.00498,
 -6758.28267,
 -5416.27653,
 3285.95561,
 7046.05022,
 7340.31738,
 5268.92475,
 -15440.35491,
 -4526.83249,
 -1387.23028,
 -1628.26653,
 11734.3552,
 -4811.57853,
 -1613.76842,
 957.38543,
 947.90634,
 -4275.53251,
 4054.29357,
 -422.67089,
 -167.85392,
 5234.02719,
 -1855.90509,
 1883.3105,
 305.74681,
 2846.60097,
 4237.82149,
 915.15342,
 -558.76465,
 -2522.14514,
 1088.64837,
 -3515.09007,
 -4773.31256,
 4466.23797,
 -2921.48411,
 -2624.1381,
 5211.74674,
 1109.300